In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import torch
import seaborn as sns
import os
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
from sklearn.model_selection import train_test_split

/home/arjun/AI_ENV/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# TODO
- See if ticket and cabin have any importance
- Try data imputation instead of fillna

# Dataset

In [2]:
loc = '/home/arjun/Desktop/Datasets/titanic'

In [3]:
df = pd.read_csv(loc + '/train.csv')

In [4]:
df = df.drop(['Name', 'Ticket', 'Cabin'], axis=1)

In [5]:
df.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S


In [6]:
df['Sex'] = df['Sex'].replace({'male': 0, 'female': 1})
df['Embarked'] = df['Embarked'].replace({'S':1, 'C':2, 'Q':3})

In [7]:
df.fillna(df.mean(), inplace=True)
df.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,0,22.0,1,0,7.2500,1.0
1,2,1,1,1,38.0,1,0,71.2833,2.0
2,3,1,3,1,26.0,0,0,7.9250,1.0
3,4,1,1,1,35.0,1,0,53.1000,1.0
4,5,0,3,0,35.0,0,0,8.0500,1.0


In [8]:
ds = torch.tensor(np.array(df))
df.shape, ds.shape

((891, 9), torch.Size([891, 9]))

In [9]:
ds.shape

torch.Size([891, 9])

In [10]:
y = ds[:,1]
X= torch.cat((ds[:,:2], ds[:,3:]), dim=-1) 

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)
X_train.shape, X_test.shape ,y_train.shape, y_test.shape

(torch.Size([712, 8]),
 torch.Size([179, 8]),
 torch.Size([712]),
 torch.Size([179]))

# Model Creation

In [12]:
num_epochs = 100
lr =.001
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [13]:
class NeuralNet(nn.Module):
    def __init__(self, inp_size, h1, h2, h3, h4, h5, out_size):
        super(NeuralNet, self).__init__()
        self.inp_size = inp_size
        self.lay1 = nn.Linear(inp_size, h1)
        self.lay2 = nn.ReLU()
        self.lay3 = nn.Linear(h1, h2)
        self.lay4 = nn.ReLU()
        self.lay5 = nn.Linear(h2, h3)
        self.lay6 = nn.ReLU()
        self.lay7 = nn.Linear(h3, h4)
        self.lay8 = nn.ReLU()
        self.lay9 = nn.Linear(h4, h5)
        self.lay10 = nn.ReLU()
        self.lay11 = nn.Linear(h5, out_size)
        
    def forward(self,x):
        out = self.lay1(x)
        out = self.lay2(out)
        out = self.lay3(out)
        out = self.lay4(out)
        out = self.lay5(out)
        out = self.lay6(out)
        out = self.lay7(out)
        out = self.lay8(out)
        out = self.lay9(out)
        out = self.lay10(out)
        out = self.lay11(out)  # We don't apply sotmax the cross entropy loss will do that for us
        return out
    
model = NeuralNet(8,30,60,120,60,30,2).to(device)

In [14]:
lossCategory = nn.CrossEntropyLoss()
optimiser = torch.optim.Adam(model.parameters(), lr = lr)

# Training Model

In [15]:
train_ds = torch.cat((X_train, y_train.reshape(-1,1)), dim=-1)  
val_ds = torch.cat((X_test, y_test.reshape(-1,1)), dim=-1)  

In [16]:
train_loader = torch.utils.data.DataLoader(dataset=train_ds, shuffle=True, batch_size=100)
val_loader = torch.utils.data.DataLoader(dataset=val_ds, shuffle=True, batch_size=100)


In [17]:
for epoch in range(num_epochs):
    avg_loss = 0
    for i,batch in enumerate(train_loader):
 
        y = batch[:,-1].to(device).long()
        X = batch[:, :-1].to(device).to(torch.float32)

        # Forward pass
        output = model(X)
        # print(output.shape, y.shape)
        
        # Backward pass
        loss = lossCategory(output, y)
        loss.backward()
        optimiser.step()
        optimiser.zero_grad()
        avg_loss += loss

    if epoch%10 == 0:    
        print(f"Epoch:{epoch+1}/{num_epochs} Loss:{avg_loss/len(train_loader)}")

Epoch:1/100 Loss:0.7615491151809692
Epoch:11/100 Loss:0.6314001083374023
Epoch:21/100 Loss:0.5986349582672119
Epoch:31/100 Loss:0.5929953455924988
Epoch:41/100 Loss:0.5804870128631592
Epoch:51/100 Loss:0.550304651260376
Epoch:61/100 Loss:0.4581625461578369
Epoch:71/100 Loss:0.327137291431427
Epoch:81/100 Loss:0.2949331998825073
Epoch:91/100 Loss:0.29083913564682007


# Checking Accuracy

In [18]:
correct = 0
tot = 0
for i,batch in enumerate(train_loader):
    
    y = batch[:,-1].to(device).long()
    X = batch[:, :-1].to(device).to(torch.float32)
    y_pred = torch.argmax(model(X), dim=1)
    s = sum(y_pred == y)
    correct += s
    tot += len(y)
    
print('Accuracy:', (correct*100/tot).item()) 

Accuracy: 93.82022857666016


In [19]:

correct = 0
tot = 0
for i,batch in enumerate(val_loader):
    y = batch[:,-1].to(device).long()
    X = batch[:, :-1].to(device).to(torch.float32)
    y_pred = torch.argmax(model(X), dim=1)
    s = sum(y_pred == y)
    correct += s
    tot += len(y)
    
print('Accuracy:', (correct*100/tot).item()) 

Accuracy: 92.7374267578125


# Running Test Dataset

In [20]:
df = pd.read_csv(loc + '/test.csv')
df = df.drop(['Name', 'Ticket', 'Cabin'], axis=1)
df['Sex'] = df['Sex'].replace({'male': 0, 'female': 1})
df['Embarked'] = df['Embarked'].replace({'S':1, 'C':2, 'Q':3})
df.fillna(df.mean(), inplace=True)
ds = torch.tensor(np.array(df))
test_loader = torch.utils.data.DataLoader(dataset=ds, shuffle=False, batch_size=100)

In [21]:
ds.shape,df.shape

(torch.Size([418, 8]), (418, 8))

In [22]:
l = []
id_ = []
for i,batch in enumerate(test_loader):
        X = batch.to(device).to(torch.float32)
        y_pred = torch.argmax(model(X), dim=1)
        # print(y_pred)

        l.extend(list(y_pred))
        # print(l)
        # id_.append(int(X[:,0].item()))
id_ = [i for i in range(892,1310 )]
l = [i.item() for i in l]
ans = pd.DataFrame({'PassengerId':id_,'Survived':l})
ans.to_csv('./submission.csv', index=False)
print("Saved new O/P")
ans

Saved new O/P


,PassengerId,Survived
0,892,1
1,893,1
2,894,1
3,895,1
4,896,1
...,...,...
413,1305,1
414,1306,1
415,1307,1
416,1308,1
